# Enunciado


3. Usando a experiência obtida na resolução dos problemas 1 e 2, e usando, ao invés  do grupo abeliano multiplicativo $\,\mathbb{F}_p^\ast\,$,  o  grupo abeliano aditivo que usou na pergunta 2,   
    1. Construa ambas as versões  IND-CPA segura e IND-CCA segura do esquema de cifra ElGamal em curvas elípticas.
    2. Construa uma implementação em curvas elípticas de um protocolo autenticado de “Oblivious Transfer” $\,\kappa$-out-of-$n\,$.

-----------

## Exercício 3.a

### **Versão IND-CPA**

#### Geração de chaves

- Escolher uma curva elíptica $E$ sobre $F_p$, com um ponto gerador $G$ de ordem prima $q$.
- A chave privada é $s∈{1,2,...,q−1}$.
- A chave pública é $(E,G,S)$, onde $S=s⋅G$.

#### Cifra

Para cifrar uma mensagem $M$ (mapeada para um ponto em $E$ na curva elíptica):

- Escolher $\omega \in {1,2,...,q−1}$ aleatoriamente.
- Calcular $\gamma=\omega⋅G$.
- Calcular $\kappa = \omega⋅S$.
- O criptograma é $(\gamma,M+\kappa)$.

#### Decifra

Recebendo a chave privada $sk=s$ e o criptograma $c=(\gamma,C)c=(\gamma,C)$:

- Calcular $\kappa=s . \gamma$ (adição de pontos na curva).
- Recuperar $M=C−\kappa$ (subtração de pontos na curva).

#### Diferenças entre usa o grupo multiplicativo $F_p^*$ e usar um grupo abeliano

No Exercício 3.A, a cifra e decifra não são iguais ao Exercício 1.A, porque as operações matemáticas mudam:

- No grupo multiplicativo $F_p^∗$, usamos exponenciação e multiplicação modular.
- Nas curvas elípticas, usamos adição de pontos.

Portanto, a cifra e decifra no Exercício 3.A devem ser adaptadas para as operações no grupo aditivo da curva elíptica. Comparando-as:

| Operação                     | Grupo Multiplicativo $ \mathbb{F}_p^* $ | Curva Elíptica $ E $ |
|------------------------------|-------------------------------------------|-------------------------|
| **Geração de Chaves**         | $ g^s\mod p $                          | $ s \cdot G $         |
| **Cifra**                     | $ \gamma = g^\omega \mod p $            | $ \gamma = \omega \cdot G $ |
|                              | $ \kappa = (g^s)^\omega \mod p $        | $ \kappa = \omega \cdot S $  |
|                              | $ \delta = m \cdot \kappa \mod p $      | $ C = M + \kappa $           |
| **Decifra**                   | $ \kappa = \gamma^s \mod p $            | $ \kappa = s \cdot \gamma $  |
|                              | $ m = \delta \cdot \kappa^{-1} \mod p $ | $ M = C - \kappa $           |

#### Implementação

In [2]:
from sage.all import EllipticCurve, GF

# Configuração da curva elíptica
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F  # Secp256k1
E = EllipticCurve(GF(p), [0, 7])  # Curva y² = x³ + 7
G = E.gens()[0]  # Gerador
q = G.order()    # Ordem do gerador

# Geração de Chaves
s = randint(1, q-1)  # Chave privada
S = s * G            # Chave pública

# Cifra
def encrypt(M, S):
    ω = randint(1, q-1)
    γ = ω * G
    κ = ω * S
    C = M + κ
    return (γ, C)

# Decifra
def decrypt(γ, C, s):
    κ = s * γ
    return C - κ

# Exemplo de uso:
M = E.random_point()  # Mapeamento da mensagem para a curva
γ, C = encrypt(M, S)
M_decrypted = decrypt(γ, C, s)
print(M_decrypted == M)

True


------

### **Transformar um  PKE-IND-CPA em um PKE-IND-CCA**

A transformação FO original constrói, a partir de $\,(E_p,D_s)\,$,  um novo esquema de cifra assimétrica $\,(E'_p,D'_s)\,$ , usando um  “hash” pseudo-aleatório $\,h\,$ de tamanho $\,\lambda\,$ e um “hash” pseudo-aleatório $\,g\,$ de tamanho $\,|x|\,$.

O algoritmo de cifra parametrizado pelos dois “hashs”  $\,h,g\,$    é 

  $$E'_{p}(x)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

O algoritmo $\,D'_{s}\,$ rejeita o criptograma se detecta algum sinal de fraude. 


$$D'_{s}(y,c)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$

## “Oblivious Transfer” $\,\kappa$-out-of-$n\,$